# Défi Grosses Data 2018

## Data analysis

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir

In [3]:
def open_and_transform(file):
    path='./data_meteo/'
    #input_file='./../data_meteo/train_1.csv'
    df = pd.read_csv(path+file, header=0, delimiter=";",decimal=",")
    #print("Dimensions:",np.shape(df))
    df['date']=df['date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
    df['insee'] = df['insee'].astype('category')
    df['mois'] = df['mois'].astype('category')
    df['ddH10_rose4'] = df['ddH10_rose4'].astype('category')
    df['ech'] = df['ech'].astype('category')
    df['flvis1SOL0'] = df['flvis1SOL0'].astype('float64')
    #print("Fichier",file," ouvert.")
    return df

In [4]:
listdir('./data_meteo/')
list_files=np.empty(36, dtype='|U12')
i=0
for fichier in listdir('./data_meteo/'):
    if 'train' in fichier:
        list_files[i]=fichier
        i=i+1

In [5]:
df_tot=pd.DataFrame()
for file in list_files:
    df_tot=pd.concat([df_tot,open_and_transform(file)])

In [6]:
df_tot=df_tot.sort_values(by=['ech','date'],ascending=True)

In [7]:
np.shape(df_tot)

(189280, 31)

<b>IMPUTATION DONNEES</b>

In [8]:
import numpy as np
import numpy.ma as ma
import random

In [11]:
df_tot.describe()
#pd.isna(df_tot['date'])

,tH2_obs,capeinsSOL0,ciwcH20,clwcH20,ffH10,flir1SOL0,fllat1SOL0,flsen1SOL0,flvis1SOL0,hcoulimSOL0,...,rrH20,tH2,tH2_VGrad_2.100,tH2_XGrad,tH2_YGrad,tpwHPA850,ux1H10,vapcSOL0,vx1H10,ech
count,189260.000000,170492.000000,1.883070e+05,184800.000000,184702.000000,188118.000000,1.882190e+05,1.881510e+05,1.881810e+05,188307.000000,...,184793.000000,184723.000000,184716.000000,184723.000000,184723.000000,188258.000000,184702.000000,188286.000000,184716.000000,189280.000000
mean,12.006981,59.169631,5.027560e-08,0.000006,3.406578,-217648.413598,-1.476294e+05,-7.151363e+04,4.136244e+05,490.318652,...,0.003668,12.143805,-0.183838,0.001031,0.000351,9.477384,1.128499,17.538598,0.732079,16.898669
std,6.263605,142.986766,1.058130e-06,0.000035,2.069121,133170.495298,2.338682e+05,2.379013e+05,6.915170e+05,505.990997,...,0.022547,6.495621,1.890783,0.002268,0.003244,4.502674,4.963149,7.752743,4.872846,9.548489
min,-7.900000,0.000000,0.000000e+00,0.000000,0.008230,-816882.554121,-1.780445e+06,-1.889882e+06,0.000000e+00,0.096693,...,0.000000,-9.776349,-9.275321,-0.022873,-0.021223,-3.477451,-22.470358,1.305881,-20.111824,1.000000
25%,7.600000,0.124246,0.000000e+00,0.000000,1.924383,-302011.786091,-1.970599e+05,-8.946848e+04,0.000000e+00,117.687765,...,0.000000,7.527034,-1.271802,-0.000110,-0.000649,6.218264,-2.001309,11.556538,-2.746925,9.000000
50%,11.600000,5.425625,0.000000e+00,0.000000,2.990316,-222379.297816,-4.894289e+04,-1.954219e+03,3.600000e-12,321.587923,...,0.000000,11.855420,-0.588375,0.000741,0.000304,9.426134,0.850176,16.337755,0.501839,17.000000
75%,16.100000,47.025052,0.000000e+00,0.000000,4.438814,-107138.680673,-2.580780e+02,4.394024e+04,5.918494e+05,709.740217,...,0.000000,16.400056,0.554582,0.001957,0.001603,12.702356,3.981172,22.293404,4.015721,25.000000
max,38.400002,2448.672118,6.457207e-05,0.000447,17.449989,78624.299681,1.857474e+05,6.836656e+05,3.206372e+06,5289.707727,...,1.754839,39.642168,11.714195,0.028023,0.031406,22.606778,29.156881,54.592355,26.574025,36.000000


tH2_obs 	Observation de la température à 2 mètres in situ- au point station (prédictant)

ech 	Echéance de validité

capeinsSOL0 	Energie potentielle convective

ciwcH20 	Fraction de glace nuageuse à 20 mètres

clwcH20 	Fraction d'eau nuageuse à 20 mètres

nH20 	Fraction nuageuse à 20 mètres

pMER0 	Pression au niveau de la mer

rr1SOL0 	Précipitation horaire au niveau du sol

rrH20 	Précipitation horaire à 20 mètres

tpwHPA850 	Température potentielle au niveau 850 hPa

ux1H10 	Rafale 1 minute du vent à 10 mètres composante zonale

vapcSOL0 	Colonne de vapeur d'eau

vx1H10 	Rafale 1 minute du vent à 10 mètres composante verticale

ddH10_rose4 	Direction du vent à 10 mètres en rose4

ffH10 	Force du vent à 10 mètres en m/s

flir1SOL0 	Flux Infra-rouge en J/m2

fllat1SOL0 	Flux de chaleur latente en J/m2

flsen1SOL0 	Flux de chaleur sensible en J/m2

flvis1SOL0 	Flux visible en J/m2

hcoulimSOL0 	Hauteur de la couche limite en mètres

huH2 	Humidité 2mètres en %

iwcSOL0 	Réservoir neige kg/m2 (équivalent en eau liquide des chutes de neige)

nbSOL0_HMoy 	Nébulosité basse (moyenne sur les 6 points de grille autour de la station) (fraction en octat du ciel occulté)

ntSOL0_HMoy 	Nébulosité totale (moyenne sur les 6 points de grille autour de la station)

tH2 	Température à 2 mètres du modèle AROME

tH2_VGrad_2.100 	Gradient vertical de température entre 2 mètres et 100 mètres

tH2_XGrad 	Gradient zonal de température à 2 mètres

tH2_YGrad

In [15]:
df_tot.shape
# Variables explicatives
df_qual=pd.get_dummies(df_tot[["mois", "insee", "ddH10_rose4", "ech"]])
#ozoneQuant=ozone[["MOCAGE","TEMPE","VentMOD","VentANG","SRMH2O","LNO2","LNO"]]
df_quant=df_tot[[
 "capeinsSOL0",
 "ciwcH20",
 "clwcH20",
 "ffH10",
 "flir1SOL0",
 "fllat1SOL0",
 "flsen1SOL0",
 "flvis1SOL0",
 "hcoulimSOL0",
 "huH2",
 "iwcSOL0",
 "nbSOL0_HMoy",
 "nH20",
 "ntSOL0_HMoy",
 "pMER0",
 "rr1SOL0",
 "rrH20",
 "tH2",
 "tH2_VGrad_2.100",
 "tH2_XGrad",
 "tH2_YGrad",
 "tpwHPA850",
 "ux1H10",
 "vapcSOL0",
 "vx1H10"]]

#faut ajouter les variables qui vont expliquer th2_obs
dfC=pd.concat([df_qual,df_quant],axis=1)

dfC.columns

Index([u'ech', u'mois_août', u'mois_avril', u'mois_décembre', u'mois_février',
       u'mois_janvier', u'mois_juillet', u'mois_juin', u'mois_mai',
       u'mois_mars', u'mois_novembre', u'mois_octobre', u'mois_septembre',
       u'insee_6088001', u'insee_31069001', u'insee_33281001',
       u'insee_35281001', u'insee_59343001', u'insee_67124001',
       u'insee_75114001', u'ddH10_rose4_1.0', u'ddH10_rose4_2.0',
       u'ddH10_rose4_3.0', u'ddH10_rose4_4.0', u'capeinsSOL0', u'ciwcH20',
       u'clwcH20', u'ffH10', u'flir1SOL0', u'fllat1SOL0', u'flsen1SOL0',
       u'flvis1SOL0', u'hcoulimSOL0', u'huH2', u'iwcSOL0', u'nbSOL0_HMoy',
       u'nH20', u'ntSOL0_HMoy', u'pMER0', u'rr1SOL0', u'rrH20', u'tH2',
       u'tH2_VGrad_2.100', u'tH2_XGrad', u'tH2_YGrad', u'tpwHPA850', u'ux1H10',
       u'vapcSOL0', u'vx1H10'],
      dtype='object')

In [17]:
dfC.dtypes

ech                  int64
mois_août            uint8
mois_avril           uint8
mois_décembre        uint8
mois_février         uint8
mois_janvier         uint8
mois_juillet         uint8
mois_juin            uint8
mois_mai             uint8
mois_mars            uint8
mois_novembre        uint8
mois_octobre         uint8
mois_septembre       uint8
insee_6088001        uint8
insee_31069001       uint8
insee_33281001       uint8
insee_35281001       uint8
insee_59343001       uint8
insee_67124001       uint8
insee_75114001       uint8
ddH10_rose4_1.0      uint8
ddH10_rose4_2.0      uint8
ddH10_rose4_3.0      uint8
ddH10_rose4_4.0      uint8
capeinsSOL0        float64
ciwcH20            float64
clwcH20            float64
ffH10              float64
flir1SOL0          float64
fllat1SOL0         float64
flsen1SOL0         float64
flvis1SOL0         float64
hcoulimSOL0        float64
huH2               float64
iwcSOL0            float64
nbSOL0_HMoy        float64
nH20               float64
n

## Imputation données avec Forêts aléatoires

In [27]:
#Données avec imputations avec Forêts aléatoires

from predictive_imputer import predictive_imputer
# Choix du mode de prévision par *k*-pp, ACP ou forêt aléatoire
imputer = predictive_imputer.PredictiveImputer(f_model="RandomForest")
# Imputation
XnaImp = imputer.fit(dfC).transform(dfC.copy())

In [240]:
from sklearn.model_selection import train_test_split
import copy
#La variable réponse
Y=df_tot[["tH2_obs"]]
XnaImpp=copy.copy(XnaImp)
# Ech d'apprentissage, Ech de test
XnaImp_train,XnaImp_test,Y_train,Y_test=train_test_split(XnaImpp,Y,test_size=0.3,random_state=11)

In [241]:
print(np.sum(np.isnan(Y_train)))
print(np.sum(np.isnan(Y_test)))
print(np.shape(Y_test), np.shape(XnaImp_test))
print(np.shape(Y_train), np.shape(XnaImp_train))


#print(XnaImp.dtypes)

#np.savetxt("foo.csv", XnaImp, delimiter=";")

#df = pd.read_csv(path+file, header=0, delimiter=";",decimal=",")

tH2_obs    14
dtype: int64
tH2_obs    6
dtype: int64
((56784, 1), (56784L, 49L))
((132496, 1), (132496L, 49L))


In [242]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor

#On remplace les données manquantes de Y par la moyenne de ce vecteur
Y_train=Y_train.fillna(Y_train.mean())
Y_test=Y_test.fillna(Y_test.mean())

# Prévisions par forêt aléatoire

# Prévision avec trous imputés
forest = RandomForestRegressor(n_estimators=200)

# Apprentissage
rfFit = forest.fit(XnaImp_train, Y_train)

1-rfFit.score(XnaImp_test, Y_test)

C:\Users\Admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.031541308493431841

1. erreur=0.031313781998513202 -> 200 arbres, 23k éch test, MSE =  1.2188613381438607
2. erreur=0.031538618660638673 ->250 arbres, size=0.3*ech, MSE = 1.2312607258699513
3. erreur = 0.031541308493431841 -> 200 arbres, 0.3*ech, MSE = 1.2313657363497386

In [178]:
print(np.sum(np.isnan(Y)))

tH2_obs    20
dtype: int64


In [243]:
# Erreur quadratique moyenne
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# prévision de l'échantillon test
Y_pred = rfFit.predict(XnaImp_test)
print("MSE =",mean_squared_error(Y_test,Y_pred))
print("R2 =",r2_score(Y_test,Y_pred))



('MSE =', 1.2313657363497386)
('R2 =', 0.96845869150656816)


In [111]:
def generate_submission_file(name, Y_PRED):
    path='./data_meteo/'
    df_template=pd.read_csv('./data_meteo/test_answer_template.csv', header=0, delimiter=";",decimal=",")
    print(np.shape(df_template.tH2_obs))
    df_template.tH2_obs=Y_PRED
    df_template.to_csv(path+name,sep=';',decimal=',',index=False)
    return 'File %s generated.' %(path+name)

In [211]:
Y_pred

array([ -0.24899999,   6.71500002,   9.51950008, ...,   8.83600004,
        18.83400002,  19.7835    ])

In [239]:
generate_submission_file("resultat1.csv", Y_pred[:21169])

## XGBoost sans imputation

In [225]:
from xgboost import XGBRegressor

X=dfC
Y=df_tot[["tH2_obs"]]
Y=Y.fillna(Y.mean())

# Ech d'apprentissage, Ech de test
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=11)

In [226]:
print(np.sum(np.isnan(Y_train)))
print(np.sum(np.isnan(Y_test)))

tH2_obs    0
dtype: int64
tH2_obs    0
dtype: int64


In [223]:
#Le modèle
model = XGBRegressor()
#Apprentissage
model.fit(X_train, Y_train)
#Erreur
1-model.score(X_test, Y_test)
#Prédiction
Y_pred1 = model.predict(X_test)

In [224]:
#Erreur quadratique moyenne
print("MSE =",mean_squared_error(Y_test,Y_pred1))
print("R2 =",r2_score(Y_test,Y_pred1))

('MSE =', 1.5989112766248088)
('R2 =', 0.95904404987680492)


## XGBoost avec imputation données

In [227]:
X=dfC
Y=df_tot[["tH2_obs"]]
Y=Y.fillna(Y.mean())

XnaImp_train,XnaImp_test,Y_train,Y_test=train_test_split(XnaImpp,Y,test_size=0.3,random_state=11)

In [238]:
#Apprentissage
model.fit(XnaImp_train, Y_train)
#Erreur
1-model.score(XnaImp_test, Y_test)
#Prédiction
Y_pred1 = model.predict(XnaImp_test, ntree_limit=100)
#on peut pas faire mieux en changeant le nombre d'arbres

#Erreur quadratique moyenne
print("MSE =",mean_squared_error(Y_test,Y_pred1))
print("R2 =",r2_score(Y_test,Y_pred1))

('MSE =', 1.5811091832212709)
('R2 =', 0.95950004869311367)


## Support Vector Machine avec imputations

In [249]:
from sklearn.svm import SVR

X=dfC
Y=df_tot[["tH2_obs"]]
Y=Y.fillna(Y.mean())

XnaImp_train,XnaImp_test,Y_train,Y_test=train_test_split(XnaImpp,Y,test_size=0.3,random_state=11)

In [ ]:
from sklearn.model_selection import GridSearchCV
param=[{"C":[0.4,0.5,0.6,0.8,1,1.4]}]
svm= GridSearchCV(SVR(),param,cv=10,n_jobs=-1)

svmOpt=svm.fit(XnaImp_train, Y_train)

# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - svmOpt.best_score_,svmOpt.best_params_))

In [ ]:
# erreur de prévision sur le test
1-svmOpt.score(X_test,Y_test)

# prévision de l'échantillon test
Y_chap = svmOpt.predict(X_test)

## Support Vector Machine sans imputations (marche pas car NaN)

In [248]:
X=dfC
Y=df_tot[["tH2_obs"]]
Y=Y.fillna(Y.mean())

XnaImp_train,XnaImp_test,Y_train,Y_test=train_test_split(XnaImpp,Y,test_size=0.3,random_state=11)

In [ ]:
svm= GridSearchCV(SVR(),param,cv=10,n_jobs=-1)

svmOpt=svm.fit(XnaImp_train, Y_train)

# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - svmOpt.best_score_,svmOpt.best_params_))

In [ ]:
# erreur de prévision sur le test
1-svmOpt.score(XnaImp_test,Y_test)

# prévision de l'échantillon test
Y_chap = svmOpt.predict(XnaImp_test)

In [45]:
print(np.sum(np.isnan(Y_train)))

Y_train.columns
np.shape(Y_train)

print(np.sum(np.isnan(Y_test)))

#np.shape(Y_test)

tH2_obs    20
dtype: int64
tH2_obs    0
dtype: int64


In [43]:
Y_train_t=Y_train.transpose()
Y_train_imp = imputer.fit(Y_train, Y).transform(Y_train.copy())

TypeError: fit() takes at least 3 arguments (2 given)

In [88]:
#Y[np.isnan(Y_train)]
#print(np.isnan(Y_train)==True)
print(np.mean(Y_train))
type(Y_train)
print(Y_train.isnull().T.any().T.sum())
print(Y_train.isnull().T.any().T)
nan_rows=Y_train[Y_train.isnull().T.any().T]
print(nan_rows)

Y_train_imp=Y_train.fillna(Y_train.mean())
print(Y_train_imp.describe())
print(Y_train_imp.isnull().T.any().T.sum())
np.shape(Y_train_imp)

tH2_obs    12.007099
dtype: float64
20
1075    False
3590    False
3012    False
638     False
325     False
4501    False
3083    False
1965    False
3406    False
1501    False
75      False
5147    False
3870    False
5627    False
532     False
1720    False
3550    False
3754    False
2271    False
6024    False
4639    False
2982    False
1427    False
2989    False
4823    False
1289    False
177     False
1099    False
2642    False
1741    False
        ...  
2624    False
5426    False
4457    False
231     False
3157    False
1468    False
4286    False
2388    False
3300    False
1842    False
6103    False
2919    False
2365    False
3589    False
1213    False
94      False
1877    False
5428    False
1322    False
2601    False
4463    False
3388    False
3472    False
5589    False
2426    False
4320    False
5350    False
3937    False
1676    False
1741    False
Length: 189080, dtype: bool
      tH2_obs
5068      NaN
5068      NaN
2490      NaN
402       NaN
402      

(189080, 1)

In [ ]:
from sklearn.metrics import mean_squared_error

Y_pred=rfFit.predict(XnaImp_test)
print("MSE=",mean_squared_error(Y_pred, Y_test))

## Imputation données avec ACP

In [134]:
#Données avec imputations avec ACP

# Choix du mode de prévision par *k*-pp, ACP ou forêt aléatoire
imputer = predictive_imputer.PredictiveImputer(f_model="PCA")
# Imputation
XnaImp = imputer.fit(dfC[:100000]).transform(dfC[:100000].copy())

In [135]:
Y=df_tot[["tH2_obs"]][:100000]

# Données sans trous
XnaImp_train,XnaImp_test,Y_train,Y_test=train_test_split(XnaImp,Y,test_size=200,random_state=11)

In [136]:
# prévision avec trous imputés
forest = RandomForestRegressor(n_estimators=100)

# apprentissage
rfFit = forest.fit(XnaImp_train,Y_train)

1-rfFit.score(XnaImp_test,Y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [121]:
Y_pred=rfFit.predict(XnaImp_test)
print("MSE=",mean_squared_error(Y_pred,Y_test))

('MSE=', 0.81138709610058868)


In [117]:
Y=df_tot[["tH2_obs"]][:500]

In [101]:
print(np.shape(Y_train))
print(np.shape(XnaImp_train))
print(XnaImp_test.shape)

type(XnaImp_train)

(300, 1)
(300, 49)
(200, 49)


numpy.ndarray

In [149]:
sum(np.isnan(XnaImp_train))
np.max(XnaImp_train)

np.sum(~ np.isfinite(XnaImp_train))

0

In [152]:
np.sum(np.isnan(Y_train)), "2eme",np.sum(~ np.isfinite(Y_train))

(tH2_obs    15
 dtype: int64, '2eme', tH2_obs    15
 dtype: int64)

In [153]:
Y=df_tot[["tH2_obs"]]
np.sum(np.isnan(Y))

tH2_obs    20
dtype: int64

In [164]:
Y_train=Y_train[np.isnan(Y_train)==True]
Y_train


,tH2_obs
2999,NaN
58,NaN
4296,NaN
206,NaN
4500,NaN
3371,NaN
3208,NaN
1536,NaN
3358,NaN
3886,NaN


Faut supprimer les valeurs manquantes de la variable observée pour faire marcher l'imputation avec l'ACP. Laisser toute la nuit l'ordinateur pour tester l'imputation avec RandomForest.
Tester également si on a du temps sur kNN